In [49]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [69]:
url_milanuncios = 'https://www.milanuncios.com/entradas-de-concierto/\
?s=coldplay%20paris%2017%20pista'
url_viagogo = 'https://www.viagogo.es'
url_viagogo2 = 'https://www.viagogo.es/Entradas-Conciertos/Indie-Rock/Coldplay-Entradas/E-8134930?qty=2'

In [37]:
def get_html(url):
	'''
	Devuelve el contenido del html en un objeto beaufitul soup
	'''
	page = requests.get(url)
	return BeautifulSoup(page.content, 'html.parser')

In [38]:
def milanuncios(url):
    anuncios = get_html(url).find_all('div', {'class':'ma-AdCard-detail'})
    ls_titulos, ls_precios, ls_descrip  = [], [], []
    for anuncio in anuncios:
        ls_titulos.append(anuncio.find('h2').get_text())
        ls_precios.append(anuncio.find('span', {'class':'ma-AdPrice-value'}).get_text())
        ls_descrip.append(anuncio.find('p', {'class':'ma-AdCardDescription-text'}).get_text())
    dictionary = {'titulo':ls_titulos, 
                  'descripcion':ls_descrip, 
                  'precio':ls_precios} 
    return pd.DataFrame.from_dict(dictionary)

In [182]:
driver = webdriver.Safari()
# entra a la pagina
driver.get(url_viagogo)
# busca 'coldplay paris 17' en el buscador de la pagina
search = driver.find_element_by_xpath('//*[@id="search"]')
search.send_keys('coldplay paris 17')
# espera a cargar y escoge el elemento que sale
# aqui podria equivocarse y coger otro concierto ? 
event = WebDriverWait(driver, 4)\
    .until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchResults"]/div[3]/div/a')))
event.click()
# abre el selector para elegir el numero de tickets
slct_qnty = WebDriverWait(driver, 20)\
    .until(EC.element_to_be_clickable((By.XPATH, '//*[@id="qtySelectionModalContent"]/div/div[1]/div[2]/div[1]')))
slct_qnty.click()
# dentro del selector queremos coger la opcion de dos entradas
# hacemos un bucle por todas las opciones y extraemos el texto 
options = driver.find_elements_by_class_name('dropdown__row')
for option in options:
    if '2 entradas' in option.text:
        if '12' not in option.text: # si hay 22, 32, 42... no funciona 
            option.click()
# damos al boton de continuar
continue_ = driver.find_element_by_xpath('//*[@id="qtySelectionModalContent"]/div/button')
continue_.click() 
# ahora scrapeamos la info de los precios 
# por todas la paginas que haya dando a 'siguiente'
ls_htmls = []
#ls_htmls.append(driver.page_source) #añadimos el primer html
# añadimos el resto dando a siguiente
next_link = 'exists'
while next_link:
    next_link = WebDriverWait(driver, 10)\
        .until(EC.presence_of_element_located((By.XPATH, '//*[@id="clientgridtable"]/div[3]/div/div/ul/li[2]/a')))
    if next_link:
        next_link.click()
        ls_htmls.append(driver.page_source) # !! el error, esta cargando siempre el mismo html
    if len(ls_htmls) > 30: # para controlar la cantidad de paginas que busca
        break
driver.close()

In [197]:
# ahora usamos BS para scrapear los htmls
#html = ls_htmls[0]
for html in ls_htmls:
    html_bs = BeautifulSoup(html, 'html.parser')
    tablon = html_bs.find('div', {'class':'f-list'}).find_all('div', {'class':'p0'})
    ls_precio = []
    for anuncio in tablon:
        seccion = anuncio.find('span', {'class':'v-title-sml'}).text.strip()
        #print(seccion)
        if 'Pista' in seccion:
            ls_precio.append(anuncio.find('span', {'class':'t-b'}).text.strip())